In [ ]:
!pip install datasets

In [10]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
import pandas as pd

print("Modules installed")


# Load your dataset from a CSV file
data = pd.read_csv("Data/code_to_code_geekforgeek.csv")

# Split your dataset into train, validation, and test sets
train_data = data.sample(frac=0.8, random_state=42)
remaining_data = data.drop(train_data.index)
valid_data = remaining_data.sample(frac=0.5, random_state=42)
test_data = remaining_data.drop(valid_data.index)


# Load the BART model and tokenizer
model_name = "facebook/bart-base"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)
print("Model loaded")

# Define a function to preprocess your dataset
def preprocess_function(examples):
    inputs = examples["input_text"]
    targets = examples["output_text"]
    return tokenizer(inputs, padding="max_length", max_length=1024, return_tensors="pt"), tokenizer(targets, padding="max_length", max_length=1024, return_tensors="pt")


# Preprocess the datasets
train_dataset = load_dataset("pandas", data_files=train_data)
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = load_dataset("pandas", data_files=valid_data)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)
test_dataset = load_dataset("pandas", data_files=test_data)
test_dataset = test_dataset.map(preprocess_function, batched=True)
print("Preprocessing complete")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./code_migration_model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    save_total_limit=2,
    num_train_epochs=3,
    eval_steps=500,
    save_steps=500,
    logging_dir="./logs",
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

print("Training Initialized")
# Fine-tune the model
trainer.train()


print("Saving model")
# Save the model
trainer.save_model()

# Optionally, evaluate the model on a test dataset
test_dataset = dataset["test"].map(preprocess_function, batched=True)
results = trainer.evaluate(test_dataset)
print(results)

# You can use the fine-tuned model for code migration


Modules installed
Model loaded


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().